In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import healpy as hp

In [3]:
kboltz=1.3806503e-23
clight=299792458.0
hplanck=6.626068e-34
TCMB = 2.72548
d2r = np.pi / 180.
pi = np.pi

lonc = 107.2
latc = 5.2

In [4]:
dirbefiles = ['../externaldata/DIRBE_ZSMA_1_256.fits', \
              '../externaldata/DIRBE_ZSMA_2_256.fits', \
              '../externaldata/DIRBE_ZSMA_3_256.fits', \
              '../externaldata/DIRBE_ZSMA_4_256.fits', \
              '../externaldata/DIRBE_ZSMA_5_256.fits', \
              '../externaldata/DIRBE_ZSMA_6_256.fits', \
              '../externaldata/DIRBE_ZSMA_7_256.fits', \
              '../externaldata/DIRBE_ZSMA_8_256.fits', \
              '../externaldata/DIRBE_ZSMA_9_256.fits', \
              '../externaldata/DIRBE_ZSMA_10_256.fits']
dirbefreqs = np.array([240, 136.36, 85.71, 61.22, 25, 12, 5, 3, 2.1428, 1.25]) * 1.e12
dirbebeams = np.array([37.6, 41.0, 39.0, 41.5, 41.0, 41.5, 42.3, 41.0, 40.4, 39.5])

In [8]:
for k, pf in enumerate(dirbefiles):
    dirbemap = hp.read_map(pf, verbose=False)
    dirbemap = hp.ud_grade(dirbemap, 512)
    
    nside = hp.get_nside(dirbemap)
    dirbemap *= 1.e6 * hp.nside2pixarea(nside)
    name = '/home/mabitbol/GBT-S140/externaldata/dirbe_smoothed_'+str(int(dirbefreqs[k]*1e-9))+'_40.fits'
    hp.write_map(name, dirbemap)

Overwriting existing file '/home/mabitbol/GBT-S140/externaldata/dirbe_smoothed_240000_40.fits'.
Overwriting existing file '/home/mabitbol/GBT-S140/externaldata/dirbe_smoothed_136360_40.fits'.
Overwriting existing file '/home/mabitbol/GBT-S140/externaldata/dirbe_smoothed_85710_40.fits'.
Overwriting existing file '/home/mabitbol/GBT-S140/externaldata/dirbe_smoothed_61220_40.fits'.
Overwriting existing file '/home/mabitbol/GBT-S140/externaldata/dirbe_smoothed_25000_40.fits'.
Overwriting existing file '/home/mabitbol/GBT-S140/externaldata/dirbe_smoothed_12000_40.fits'.
Overwriting existing file '/home/mabitbol/GBT-S140/externaldata/dirbe_smoothed_5000_40.fits'.
Overwriting existing file '/home/mabitbol/GBT-S140/externaldata/dirbe_smoothed_3000_40.fits'.
Overwriting existing file '/home/mabitbol/GBT-S140/externaldata/dirbe_smoothed_2142_40.fits'.
Overwriting existing file '/home/mabitbol/GBT-S140/externaldata/dirbe_smoothed_1250_40.fits'.


In [4]:
for k, pf in enumerate(wmapfiles):
    wmapmap = hp.read_map(pf, verbose=False) * 1.e-3
    nside = hp.get_nside(wmapmap)
    nu = wmapfreqs[k]
    
    X = hplanck * nu / (kboltz * TCMB)
    kthermo_to_intensity = 2. * kboltz * (nu / clight)**2 * (X**2 * np.exp(X)) / (np.exp(X) - 1.)**2
    
    wmapmap *= kthermo_to_intensity * hp.nside2pixarea(nside) * 1.e26
    name = '/home/mabitbol/GBT-S140/externaldata/wmap_smoothed_'+str(int(nu*1e-9))+'.fits'
    hp.write_map(name, wmapmap)

Overwriting existing file '/home/mabitbol/GBT-S140/externaldata/wmap_smoothed_22.fits'.
Overwriting existing file '/home/mabitbol/GBT-S140/externaldata/wmap_smoothed_32.fits'.
Overwriting existing file '/home/mabitbol/GBT-S140/externaldata/wmap_smoothed_40.fits'.
Overwriting existing file '/home/mabitbol/GBT-S140/externaldata/wmap_smoothed_60.fits'.
Overwriting existing file '/home/mabitbol/GBT-S140/externaldata/wmap_smoothed_93.fits'.


In [4]:
planckfiles = ['../externaldata/LFI_SkyMap_030-field-IQU_1024_R2.01_full.fits', \
              '../externaldata/LFI_SkyMap_044-field-IQU_1024_R2.01_full.fits', \
              '../externaldata/LFI_SkyMap_070-field-IQU_1024_R2.01_full.fits', \
              '../externaldata/HFI_SkyMap_100-field-IQU_2048_R2.02_full.fits', \
              '../externaldata/HFI_SkyMap_143-field-IQU_2048_R2.02_full.fits', \
              '../externaldata/HFI_SkyMap_217-field-IQU_2048_R2.02_full.fits', \
              '../externaldata/HFI_SkyMap_353-field-IQU_2048_R2.02_full.fits', \
              '../externaldata/HFI_SkyMap_545-field-Int_2048_R2.02_full.fits', \
              '../externaldata/HFI_SkyMap_857-field-Int_2048_R2.02_full.fits']
planckfreqs = np.array([28.4, 44.1, 70.4, 100., 143., 217., 353., 545., 857.]) * 1e9
planckbeams = np.array([32.3, 27.1, 13.3, 9.7, 7.3, 5., 4.8, 4.7, 4.3])

In [7]:
k = 3
pf = planckfiles[k]
planckmap = hp.read_map(pf, verbose=False)
newbeam = np.sqrt(40.**2 - planckbeams[k]**2) / 60.
planckmap = hp.ud_grade(hp.smoothing(planckmap, fwhm=newbeam*d2r), 512)

nside = hp.get_nside(planckmap)
nu = planckfreqs[k]

X = hplanck * nu / (kboltz * TCMB)
kthermo_to_intensity = 2. * kboltz * (nu / clight)**2 * (X**2 * np.exp(X)) / (np.exp(X) - 1.)**2
if nu < 400e9:
    planckmap *= kthermo_to_intensity * hp.nside2pixarea(nside) * 1.e26
else:
    planckmap *= 1e6 * hp.nside2pixarea(nside)

name = '/home/mabitbol/GBT-S140/externaldata/planck_smoothed_'+str(int(planckfreqs[k]*1e-9))+'_40.fits'
hp.write_map(name, planckmap)

Sigma is 16.479415 arcmin (0.004794 rad) 
-> fwhm is 38.806056 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin


In [12]:
for k, pf in enumerate(planckfiles):
    planckmap = hp.read_map(pf, verbose=False)
    newbeam = np.sqrt(40.**2 - planckbeams[k]**2) / 60.
    planckmap = hp.ud_grade(hp.smoothing(planckmap, fwhm=newbeam*d2r), 512)

    nside = hp.get_nside(planckmap)
    nu = planckfreqs[k]
    
    X = hplanck * nu / (kboltz * TCMB)
    kthermo_to_intensity = 2. * kboltz * (nu / clight)**2 * (X**2 * np.exp(X)) / (np.exp(X) - 1.)**2
    if nu < 400e9:
        planckmap *= kthermo_to_intensity * hp.nside2pixarea(nside) * 1.e26
    else:
        planckmap *= 1e6 * hp.nside2pixarea(nside)
        
    name = '/home/mabitbol/GBT-S140/externaldata/planck_smoothed_'+str(int(planckfreqs[k]*1e-9))+'_40.fits'
    hp.write_map(name, planckmap)

Sigma is 10.019748 arcmin (0.002915 rad) 
-> fwhm is 23.594703 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 12.493911 arcmin (0.003634 rad) 
-> fwhm is 29.420911 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 16.019963 arcmin (0.004660 rad) 
-> fwhm is 37.724130 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 16.701163 arcmin (0.004858 rad) 
-> fwhm is 39.328234 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 16.853207 arcmin (0.004902 rad) 
-> fwhm is 39.686270 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 16.863690 arcmin (0.004905 rad) 
-> fwhm is 39.710956 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 16.868769 arcmin (0.004907 rad) 
-> fwhm is 39.722915 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 16.888001 arcmin (0.00491

In [13]:

for k, pf in enumerate(ph.planckfiles):
    planckmap = hp.read_map(pf, verbose=False)
    newbeam = np.sqrt(60.**2 - ph.planckbeams[k]**2) / 60.
    planckmap = hp.ud_grade(hp.smoothing(planckmap, fwhm=newbeam*ph.d2r), 512)
    name = '/home/mabitbol/GBT-S140/externaldata/planck_smoothed_'+str(int(ph.planckfreqs[k]*1e-9))+'.fits'
    hp.write_map(name, planckmap)

In [13]:
def smoothingscript():
    for k, pf in enumerate(ph.planckfiles):
        planckmap = hp.read_map(pf, verbose=False)
        newbeam = np.sqrt(60.**2 - ph.planckbeams[k]**2) / 60.
        planckmap = hp.ud_grade(hp.smoothing(planckmap, fwhm=newbeam*ph.d2r), 512)
        name = '/home/mabitbol/GBT-S140/externaldata/planck_smoothed_'+str(int(ph.planckfreqs[k]*1e-9))+'.fits'
        hp.write_map(name, planckmap)